In [23]:
import datetime, requests

fecha_actual = datetime.date.today()
un_dia       = datetime.timedelta(days=1)

ANIO_INICIO = fecha_actual.year - 4
MES_INICIO  = fecha_actual.month
DIA_INICIO  = fecha_actual.day 

fecha_inicio = datetime.date(ANIO_INICIO, MES_INICIO, DIA_INICIO)

In [24]:
URL_BASE = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/' 

resultados = list()

while fecha_inicio <= fecha_actual:
    resultados.append('El archivo no existe.' in requests.get(
        URL_BASE + 
        f'datohorario{fecha_inicio.year}{fecha_inicio.month:02}{fecha_inicio.day:02}'))
    fecha_inicio += un_dia

print(any(resultados))

False


# TODO

In [ ]:
for line in r.iter_lines(decode_unicode='utf8'):
    if line.startswith(('F', ' ')):
        continue
    print(line)

In [ ]:
with open('datest.csv', 'w') as f:
    for line in r.iter_lines(decode_unicode='utf8'):
        if line.startswith(('F', ' ')):
            continue
        f.write(','.join(f'{line.strip()}\n').split(maxsplit=7))